In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
import pickle
import time
import cmath
import itertools
import math

2022-05-27 01:45:36.662564: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-27 01:45:36.662581: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## ODE-side stuff

In [2]:
def forward_euler(ddt, u0, T, *args):
    u = np.empty((T.size, u0.size))
    u[0] = u0
    for i in range(1, T.size):
        u[i] = u[i-1] + (T[i] - T[i-1]) * ddt(u[i-1], T[i-1], *args)
    return u


def F_all(eom, jac, u, M, params):

    ''' solves system. u is the state vector composed
        by the variables. J is the Jacobian and M are the
        Gram-Schmidt Lyapunov vectors'''
    
    dqdt  = eom(u, params)
    J     = jac(u, params)
    dMdt  = np.dot(J, M)

    return dqdt, dMdt


def qr_factorization(A):
    m, n = A.shape
    Q = np.zeros((m, n))
    R = np.zeros((n, n))

    for j in range(n):
        v = A[:, j].copy()

        for i in range(j):
            q = Q[:, i]
            R[i, j] = q.dot(v)
            v = v - R[i, j] * q

        norm = np.linalg.norm(v)
        Q[:, j] = v / norm
        R[j, j] = norm
    return Q, R

def RK4var(f, eom, jac, x, dt, pf, mat):
    
    K1, M1 = f(eom, jac, x, mat, pf)        
    K2, M2 = f(eom, jac, x + dt*K1/2.0, mat + dt*M1/2.0, pf)
    K3, M3 = f(eom, jac, x + dt*K2/2.0, mat + dt*M2/2.0, pf)
    K4, M4 = f(eom, jac, x + dt*K3, mat + dt*M3, pf)
    
    A = np.array(dt * (K1/2.0 + K2 + K3 + K4/2.0) / 3.0)
    B = np.array(dt * (M1/2.0 + M2 + M3 + M4/2.0) / 3.0)
    return A, B

def RK4(ddt, u0, T, *args):
    u = np.empty((T.size, u0.size))
    u[0] = u0
    
    for i in range(1, T.size):    
        K1 = ddt(u[i-1], T[i-1]            , *args)
        K2 = ddt(u[i-1] + dt*K1/2.0, T[i-1], *args)
        K3 = ddt(u[i-1] + dt*K2/2.0, T[i-1], *args)
        K4 = ddt(u[i-1] + dt*K3, T[i-1]    , *args)
        
        u[i] = u[i-1] + np.array((T[i] - T[i-1]) * (K1/2.0 + K2 + K3 + K4/2.0) / 3.0)
        
    return u




def solve_ode(N, dt, u0, params=[8/3, 28, 10]):
    """
        Solves the ODEs for N time steps starting from u0.
        Returned values are normalized.

        Args:
            N: number of time steps
            dt: timestep
            u0: initial condition
            params: parameters for ODE
        Returns:
            normalized time series of shape (N+1, u0.size)
    """

    T = np.arange(N+1) * dt    
    U = RK4(ddt, u0, T, params)

    return U

def lorenz63(u, params):    
    beta, rho, sigma = params
    x, y, z = u
    return np.array([sigma*(y-x), x*(rho-z)-y, x*y-beta*z])

def rossler(u, params):    
    a, b, c = params
    x, y, z = u
    return np.array([ -y -z, x + a*y, b + z*(x-c)])

def cdv(u, params):
    
    x1, x2, x3, x4, x5, x6 = u
    x1star, x4star, C, beta, gamma, b = params    
    
    def alpha_m(m,b):
        return (8.*math.sqrt(2.)/math.pi)*(m*m/(4*m*m-1))*((b*b +m*m -1)/(b*b+m*m))
    def beta_m(m,b):
        return beta*b*b/(b*b+m*m)
    def gamma_m(m,b):
        return gamma*(4*m*m*m/(4*m*m-1))*(math.sqrt(2)*b/(math.pi*(b*b + m*m)))
    def gamma_m_tld(m,b):
        return gamma*(4*m/(4*m*m-1))*(math.sqrt(2)*b/math.pi)
    def delta(m,b):
        return (64.*math.sqrt(2.)/(15.*math.pi))*((b*b -m*m+1)/(b*b+m*m))

    #epsilon = 16.*math.sqrt(2)/(5*math.pi)
    epsilon = 1.44050610585137
    
    x1dot =  gamma_m_tld(1,b)*x3 - C*(x1 - x1star)
    x4dot =  gamma_m_tld(2,b)*x6 - C*(x4 - x4star) + epsilon*(x2*x6 - x3*x5)
    
    x2dot = -(alpha_m(1,b)*x1 - beta_m(1,b))*x3 - C*x2 -delta(1,b)*x4*x6
    x5dot = -(alpha_m(2,b)*x1 - beta_m(2,b))*x6 - C*x5 -delta(2,b)*x4*x3
    
    x3dot =  (alpha_m(1,b)*x1 - beta_m(1,b))*x2 - gamma_m(1,b)*x1 - C*x3 + delta(1,b)*x4*x5
    x6dot =  (alpha_m(2,b)*x1 - beta_m(2,b))*x5 - gamma_m(2,b)*x4 - C*x6 + delta(2,b)*x4*x2
    
    return np.array([ x1dot, x2dot, x3dot, x4dot, x5dot, x6dot])
        

def cdvjac(u, params):
    
    x1, x2, x3, x4, x5, x6 = u
    x1star, x4star, C, beta, gamma, b = params
    
    def alpha_m(m,b):
        return (8.*math.sqrt(2.)/math.pi)*(m*m/(4*m*m-1))*((b*b +m*m -1)/(b*b+m*m))
    def beta_m(m,b):
        return beta*b*b/(b*b+m*m)
    def gamma_m(m,b):
        return gamma*(4*m*m*m/(4*m*m-1))*(math.sqrt(2)*b/(math.pi*(b*b + m*m)))
    def gamma_m_tld(m,b):
        return gamma*(4*m/(4*m*m-1))*(math.sqrt(2)*b/math.pi)
    def delta(m,b):
        return (64.*math.sqrt(2.)/(15.*math.pi))*((b*b -m*m+1)/(b*b+m*m))

    #epsilon = 16.*math.sqrt(2)/(5*math.pi)
    epsilon = 1.44050610585137
    
    dx1dot = np.array([-C, 0., gamma_m_tld(1,b), 0., 0., 0.])
    dx2dot = np.array([-alpha_m(1,b)*x3, -C, -alpha_m(1,b)*x1 + beta_m(1,b), -delta(1,b)*x6, 0.,-delta(1,b)*x4])
    dx3dot = np.array([alpha_m(1,b)*x2-gamma_m(1,b), alpha_m(1,b)*x1 - beta_m(1,b), -C, delta(1,b)*x5,delta(1,b)*x4, 0.  ])
    dx4dot = np.array([ 0., epsilon*x6, -epsilon*x5, -C, -epsilon*x3, gamma_m_tld(2,b)+epsilon*x2 ])
    dx5dot = np.array([ -alpha_m(2,b)*x6, 0., -delta(2,b)*x4, -delta(2,b)*x3, -C, -alpha_m(2,b)*x1 +beta_m(2,b)])   
    dx6dot = np.array([ alpha_m(2,b)*x5, delta(2,b)*x4, 0, -gamma_m(2,b) + delta(2,b)*x2, alpha_m(2,b)*x1 - beta_m(2,b), -C])
        
    return np.array([ dx1dot, dx2dot, dx3dot, dx4dot, dx5dot, dx6dot])
    
def l63jac(u, params):
    beta, rho, sigma = params # Unpack the constants vector
    x, y, z          = u  # Unpack the state vector

    #Jacobian
    J = np.array([[-sigma, sigma,     0],
                  [ rho-z,    -1,    -x],
                  [     y,     x, -beta]])
    return J

def rosjac(u, params):
    a, b, c = params # Unpack the constants vector
    x, y, z = u  # Unpack the state vector

    #Jacobian
    J = np.array([[0, -1,  -1],
                  [1,  a,   0],
                  [z,  0, x-c]])
    return J

def lorenz96(x, p):
    return np.roll(x,1) * (np.roll(x,-1) - np.roll(x,2)) - x + p

def l96jac(x, p):
    D = len(x)
    J = np.zeros((D,D), dtype='float')
    for i in range(D):
        J[i,(i-1)%D] =  x[(i+1)%D] - x[(i-2)%D]
        J[i,(i+1)%D] =  x[(i-1)%D]
        J[i,(i-2)%D] = -x[(i-1)%D]
        J[i,i] = -1.0
    return J

def solve_ode_LEs(system, N, Ntherm, dt, u0, saveclvs, params, fname, subspace_LEs_indeces, norm_time=1):
    """
        Solves the ODEs for N time steps starting from u0.
        Additionally it computes the Lyapunov spectrum and CLVs
        Returned values are normalized.

        Args:
            system: str that defines the dynamical system
            N: number of time steps
            Ntherm: number of time steps for initial transient.
            dt: timestep
            u0: initial condition
            saveclvs: flag to select if to save full CLVs or do a consistency check
            norm_time: modulo of QR decomposition
            params: parameters for ODE
            subspace_LEs_indeces: for high dimensional systems D>3 the ordering of CLVs in subspaces.
        Returns:
            normalized time series of shape (N+1, u0.size)
    """
        
    
    print('Dynamical system:',system)
    if system=='lorenz96':
        eom = lorenz96
        jac = l96jac
    if system=='lorenz63':
        eom = lorenz63        
        jac = l63jac
    if system=='rossler':
        eom = rossler
        jac = rosjac
    if system=='cdv':
        eom = cdv
        jac = cdvjac
    
    #Timesteps in test set
    N_test = N - Ntherm    
        
    T = np.arange(N+1) * dt    
    Ttest = np.arange(1,int((N_test)/norm_time)+1) * dt * norm_time
    xt = u0
    Xt = np.empty((T.size, u0.size))
    Xt[0] = u0
    
    N_test_norm = int(N_test/norm_time)
    
    # Lyapunov Exponents timeseries
    LE   = np.zeros((N_test_norm,dim))    
    # finite-time Lyapunov Exponents timeseries
    FTLE = np.zeros((N_test_norm,dim))
    # Q matrix recorded in time
    QQ_t = np.zeros((dim,dim,N_test_norm))
    # R matrix recorded in time
    RR_t = np.zeros((dim,dim,N_test_norm))
    
    #set random orthonormal Lyapunov vectors (GSVs) 
    U    = scipy.linalg.orth(np.random.rand(dim,dim))#np.eye(dim)
    Q, R = qr_factorization(U)
    U    = Q[:,:dim]    
    
    indx = 0    
    for i in range(1, T.size):        
        xt0       = xt
        xn, Mtemp = RK4var(F_all, eom, jac, xt, dt, params, U)        
        xt       += xn
        U        += Mtemp             

        if i%norm_time==0:            
            
            Q, R = qr_factorization(U)
            U    = Q[:,:dim]
            if i > Ntherm:                
                QQ_t[:,:,indx] = Q
                RR_t[:,:,indx] = R
                LE[indx]       = np.abs(np.diag(R))
                Jacobian       = jac(xt, params)                
                FTLE[indx]     = (1./dt)*np.log(LE[indx])
                indx          += 1

        Xt[i] = xt
    
    
    LEs = np.cumsum(np.log(LE[:]),axis=0) / np.tile(Ttest[:],(dim,1)).T    
    
    #Calculation of CLVs
    if check_clv == 1: 
        print("Calculate CLVs")                
        CLV_calculation(system,params,None,None,QQ_t,RR_t,dim,\
                        Ntherm,dim,False,dt*norm_time,fname,\
                        Xt, None, norm_time, saveclvs,LEs,\
                        FTLE, subspace_LEs_indeces)
    #avFTLES = np.average(FTLE,axis=0)
    #print("average FTLEs",avFTLES)
    
    return Xt, LEs


In [3]:
## ESN with bias architecture

def step(x_pre, u, sigma_in, rho):
    """ Advances one ESN time step.
        Args:
            x_pre: reservoir state
            u: input
        Returns:
            new augmented state (new state with bias_out appended)
    """
    # input is normalized and input bias added
    u_augmented = np.hstack((u/norm, bias_in))
    # reservoir update
    x_post      = np.tanh(Win.dot(u_augmented*sigma_in) + W.dot(rho*x_pre))
    # output bias added
    x_augmented = np.concatenate((x_post, bias_out))

    return x_augmented


def const_jacobian(Wout):    
    dfdu = np.r_[np.diag(sigma_in/norm),[np.zeros(dim)]]
    d    = Win.dot(dfdu) 
    c    = np.matmul(d,Wout[:N_units,:].T) 

    return c, W.dot(np.diag(np.ones(N_units)*rho))

def open_loop(U, x0, sigma_in, rho):
    """ Advances ESN in open-loop.
        Args:
            U: input time series
            x0: initial reservoir state
        Returns:
            time series of augmented reservoir states
    """
    N       = U.shape[0]
    N_units = x0.shape[0]
    Xa      = np.empty((N+1, N_units+1))
    Xa[0]   = np.concatenate((x0,bias_out))
    for i in np.arange(1,N+1):
        Xa[i] = step(Xa[i-1,:N_units], U[i-1], sigma_in, rho)
        
    return Xa

def closed_loop(N, x0, Wout, sigma_in, rho):
    """ Advances ESN in closed-loop.
        Args:
            N: number of time steps
            x0: initial reservoir state
            Wout: output matrix
        Returns:
            time series of prediction
            final augmented reservoir state
    """
    
    xa    = x0.copy()
    Yh    = np.empty((N+1, dim))
    Yh[0] = np.dot(xa, Wout)    
    
    for i in np.arange(1,N+1):
        xa    = step(xa[:N_units], Yh[i-1], sigma_in, rho)
        Yh[i] = np.dot(xa, Wout) 

    return Yh, xa

def closed_loop_jacobian(N, x0, Wout, sigma_in, rho, norm_time, fname,\
                         saveclvs, system, subspace_LEs_indeces, params):
    """ Advances ESN in closed-loop and calculates the ESN Jacobian and Lyapunov exponents and vectors.
        Args:
            N: number of time steps
            x0: initial reservoir state
            Wout: output matrix
        Returns:
            time series of prediction
            final augmented reservoir state
    """
    # Discard 1/10 of initial test set as a transient for the 
    # tangent vectors to relax on the attractor.
    Ntransient = int(N/10)
    
    N_test = N - Ntransient    
    Ttot  = np.arange(int(N_test/norm_time)) * dt * norm_time
    
    N_test_norm = int(N_test/norm_time)
    
    # Lyapunov Exponents timeseries
    LE = np.zeros((N_test_norm,dim))    
    # finite-time Backward Lyapunov Exponents timeseries
    FTLE = np.zeros((N_test_norm,dim))
    # Q matrix recorded in time    
    QQ_t = np.zeros((N_units,dim,N_test_norm))
    # R matrix recorded in time
    RR_t = np.zeros((dim,dim,N_test_norm))
        
    xa    = x0.copy()
    Yh    = np.empty((N, dim))    
    xat   = np.empty((N, N_units))
    xat[0]= xa[:N_units]
    Yh[0] = np.dot(xa, Wout)
    
    const_jac_a, const_jac_b = const_jacobian(Wout)
    
    #Initialize the GSVs
    U    = scipy.linalg.orth(np.random.rand(N_units,dim))    
    Q, R = qr_factorization(U)
    U    = Q[:,:dim].copy()   
    
    
    for i in np.arange(1,Ntransient):
        xa    = step(xa[:N_units], Yh[i-1], sigma_in, rho)
        Yh[i] = np.dot(xa, Wout)
        xat[i]= xa[:N_units].copy()

        diag_mat = np.diag(1 - xa[:N_units]*xa[:N_units])
        jacobian = np.matmul(diag_mat,const_jac_a) + np.matmul(diag_mat,const_jac_b)
        U        = np.matmul(jacobian, U)
        
        if i % norm_time == 0:            
            Q, R = qr_factorization(U)
            U    = Q[:,:dim].copy()

    indx = 0
    for i in np.arange(Ntransient,N):
        
        xa    = step(xa[:N_units], Yh[i-1], sigma_in, rho)
        Yh[i] = np.dot(xa, Wout) 
        xat[i]= xa[:N_units].copy()

        diag_mat = np.diag(1 - xa[:N_units]*xa[:N_units])
        jacobian = np.matmul(diag_mat,const_jac_a) + np.matmul(diag_mat,const_jac_b)        
        U        = np.matmul(jacobian, U)
        
        if i % norm_time == 0:            
            Q, R  = qr_factorization(U)
            U     = Q[:,:dim].copy()
            
            RR_t[:,:,indx] = R.copy()
            QQ_t[:,:,indx] = Q.copy()
            LE[indx]       = np.abs(np.diag(R[:dim,:dim]))
            FTLE[indx]     = (1./dt)*np.log(LE[indx])
            indx          +=1

            if i%20000==0:
                print('Inside closed loop i=',i)    
    LEs = np.cumsum(np.log(LE[1:]),axis=0) / np.tile(Ttot[1:],(dim,1)).T
    print('ESN Lyapunov exponents: ',LEs[-1])
    

    #Calculation of CLVs
    if check_clv == 1: 
        print("Calculate CLVs")        
        CLV_calculation(system,params,const_jac_a, const_jac_b,QQ_t,RR_t,dim,Ntransient,\
                        N_units,False,dt*norm_time,fname, Yh, xat, norm_time,\
                        saveclvs, LEs, FTLE, subspace_LEs_indeces)
    
    #avFTLES = np.average(FTLE,axis=0)
    #print("average FTLEs",avFTLES)
    
    return Yh, xa, LEs

def train_n(U_washout, U_train, Y_train, tikh, sigma_in, rho):
    """ Trains ESN.
        Args:
            U_washout: washout input time series
            U_train: training input time series
            tikh: Tikhonov factor
        Returns:
            time series of augmented reservoir states
            optimal output matrix
    """
        
    ## initial washout phase
    xf = open_loop(U_washout, np.zeros(N_units), sigma_in, rho)[-1,:N_units]
    
    ## splitting training in N_splits to save memory
    LHS = 0
    RHS = 0
    N_len = (U_train.shape[0]-1)//N_splits
    
    for ii in range(N_splits):
        ## open-loop train phase
        t1  = time.time()
        Xa1 = open_loop(U_train[ii*N_len:(ii+1)*N_len], xf, sigma_in, rho)[1:]
        xf  = Xa1[-1,:N_units].copy()
        if ii == 0 and k==0: print('open_loop time:', (time.time()-t1)*N_splits)
        
        ##computing the matrices for the linear system
        t1  = time.time()   
        LHS += np.dot(Xa1.T, Xa1) 
        RHS += np.dot(Xa1.T, Y_train[ii*N_len:(ii+1)*N_len])        
        if ii == 0 and k==0: print('matrix multiplication time:', (time.time()-t1)*N_splits)
    
    # to cover the last part of the data that didn't make into the even splits
    if N_splits > 1:
        Xa1 = open_loop(U_train[(ii+1)*N_len:], xf, sigma_in, rho)[1:]
        LHS += np.dot(Xa1.T, Xa1) 
        RHS += np.dot(Xa1.T, Y_train[(ii+1)*N_len:])
    
    Wout = np.empty((len(tikh),N_units+1,dim))
    
    # solve linear system for different Tikhonov
    for j in range(len(tikh)):
        if j == 0: #add tikhonov to the diagonal (fast way that requires less memory)
            LHS.ravel()[::LHS.shape[1]+1] += tikh[j]
        else:
            LHS.ravel()[::LHS.shape[1]+1] += tikh[j] - tikh[j-1]
        
        #solve linear system
        t1  = time.time()
        Wout[j] = np.linalg.solve(LHS, RHS)

        if j==0 and k==0: print('linear system time:', (time.time() - t1)*len(tikh))

    return Wout, LHS, RHS

def train_save_n(U_washout, U_train, Y_train, tikh, sigma_in, rho, noise):
    """ Trains ESN.
        Args:
            U_washout: washout input time series
            U_train: training input time series
            tikh: Tikhonov factor
        Returns:
            time series of augmented reservoir states
            optimal output matrix
    """

    ## washout phase
    xf    = open_loop(U_washout, np.zeros(N_units), sigma_in, rho)[-1,:N_units]
    
    LHS   = 0
    RHS   = 0
    N_len = (U_train.shape[0]-1)//N_splits
    
    for ii in range(N_splits):
        t1  = time.time()
        ## open-loop train phase
        Xa1 = open_loop(U_train[ii*N_len:(ii+1)*N_len], xf, sigma_in, rho)[1:]
        xf  = Xa1[-1,:N_units].copy()

        t1  = time.time()   
        LHS += np.dot(Xa1.T, Xa1) 
        RHS += np.dot(Xa1.T, Y_train[ii*N_len:(ii+1)*N_len])
                    
    if N_splits > 1:# to cover the last part of the data that didn't make into the even splits
        Xa1 = open_loop(U_train[(ii+1)*N_len:], xf, sigma_in, rho)[1:]
        LHS += np.dot(Xa1.T, Xa1) 
        RHS += np.dot(Xa1.T, Y_train[(ii+1)*N_len:])

    LHS.ravel()[::LHS.shape[1]+1] += tikh
    
    Wout = np.linalg.solve(LHS, RHS)
    
    return Wout

## Covariant Lyapunov Vectors Calculation

In [4]:
def normalize(M):
    ''' Normalizes columns of M individually '''
    nM = np.zeros(M.shape) # normalized matrix
    nV = np.zeros(np.shape(M)[1]) # norms of columns

    for i in range(M.shape[1]):
        nV[i] = scipy.linalg.norm(M[:,i])
        nM[:,i] = M[:,i] / nV[i]

    return nM, nV

def timeseriesdot(x,y, multype): 
    tsdot = np.einsum(multype,x,y.T) #Einstein summation. Index i is time.
    return tsdot

def vec_normalize(vec,timeaxis):
    #normalize a vector within its timeseries
    timetot = np.shape(vec)[timeaxis]
    for t in range(timetot):        
        vec[t,:] = vec[t,:] / np.linalg.norm(vec[t,:])
    return vec

def subspace_angles(clv, timeaxis, index):
    #calculate principal angles between subspaces    
    timetot = np.shape(clv)[timeaxis]      
    thetas = np.zeros((timetot,3))
    
    #Nv_un and clvs of the unstable expanding subspace
    Nv_un = index[0]
    CLV_un = clv[:,0:Nv_un,:]
    print('CLV_un shape', CLV_un.shape)
    pos_clvs = Nv_un
    
    #Nv_nu and clvs of the neutral subspace
    Nv_nu = index[1]
    CLV_nu = clv[:,Nv_un:Nv_un+Nv_nu,:]
    print('CLV_nu shape', CLV_nu.shape)
    neut_clvs = Nv_nu
    
    #clvs of the stable decaying subspace
    CLV_st = clv[:,Nv_un+Nv_nu:,:]
    print('CLV_st shape', CLV_st.shape)
    neg_clvs = np.shape(CLV_st)[1]
    
    for t in range(timetot):        
        thetas[t,0] = np.rad2deg(scipy.linalg.subspace_angles(CLV_un[:,:,t],CLV_nu[:,:,t]))[-1]
        thetas[t,1] = np.rad2deg(scipy.linalg.subspace_angles(CLV_un[:,:,t],CLV_st[:,:,t]))[-1]
        thetas[t,2] = np.rad2deg(scipy.linalg.subspace_angles(CLV_nu[:,:,t],CLV_st[:,:,t]))[-1]
    
    return thetas

def CLV_angles(clv, NLy):
    #calculate angles between CLVs        
    costhetas = np.zeros((clv[:,0,:].shape[1],NLy))
    count = 0
    for subset in itertools.combinations(np.arange(NLy), NLy-1):
        index1 = subset[0]
        index2 = subset[1]        
        #For principal angles take the absolute of the dot product        
        costhetas[:,count] = np.absolute(timeseriesdot(clv[:,index1,:],clv[:,index2,:],'ij,ji->j'))
        count+=1
    thetas = 180. * np.arccos(costhetas) / math.pi
    
    return thetas

def CLV_calculation(system,params,const_jac_a, const_jac_b,QQ,RR,NLy,Ntherm,\
                    U_dim,adj,dt,fname,state, xa, norm_time,\
                    saveclvs, LEs, FTLE, subspace_LEs_indeces):
    
    tly = np.shape(QQ)[-1]
    su = int(tly / 10)
    sd = int(tly / 10)
    s  = su              # index of spinup time
    e  = tly+1 - sd      # index of spindown time
    tau = int(dt/dt)     # time for finite-time lyapunov exponents

    #Calculation of CLVs
    C = np.zeros((NLy,NLy,tly))        # coordinates of CLVs in local GS vector basis
    D = np.zeros((NLy,tly))            # diagonal matrix with CLV growth factors
    V = np.zeros((U_dim,NLy,tly))      # coordinates of CLVs in physical space (each column is a vector)


    # FTCLE: Finite-time lyapunov exponents along CLVs
    il  = np.zeros((NLy,tly+1)) 

    # Dynamical system
    if system=='lorenz96':
        eom = lorenz96
        jac = l96jac
    if system=='lorenz63':
        eom = lorenz63
        jac = l63jac
    if system=='rossler':
        eom = rossler
        jac = rosjac
    if system=='cdv':
        eom = cdv
        jac = cdvjac

    # initialise components to I
    C[:,:,-1] = np.eye(NLy)
    D[:,-1]   = np.ones(NLy)
    V[:,:,-1] = np.dot(np.real(QQ[:,:,-1]), C[:,:,-1])

    for i in reversed(range( tly-1 ) ):
        C[:,:,i], D[:,i] = normalize(scipy.linalg.solve_triangular(np.real(RR[:,:,i]), C[:,:,i+1]))
        V[:,:,i]         = np.dot(np.real(QQ[:,:,i]), C[:,:,i])

    
    # Computes the FTCLEs
    for j in 1+np.arange(s, e): #time loop
        il[:,j] = -(1./dt)*np.log(D[:,j])
   
    hf = h5py.File(fname+'.h5','w')
    if saveclvs == 'all':
        hf.create_dataset('CLV',      data=V)
        hf.create_dataset('FTCLE',    data=il)
        hf.create_dataset('FTLE',     data=FTLE)
        hf.create_dataset('LEs',      data=LEs)
        hf.create_dataset('state',    data=state)        
        hf.create_dataset('Ntherm',   data=Ntherm)
        hf.create_dataset('norm_time',data=norm_time)
    elif saveclvs == 'angles':
        #normalize CLVs before measuring their angles.
        timetot = np.shape(V)[-1]
        for i in range(NLy):
            for t in range(timetot-1):
                V[:,i,t] = V[:,i,t] / np.linalg.norm(V[:,i,t])

        if system == 'lorenz96' or system == 'cdv':
            thetas_clv = subspace_angles(V, -1, subspace_LEs_indeces)
        else:
            thetas_clv = CLV_angles(V, NLy)        

        
        hf.create_dataset('thetas_clv', data=thetas_clv)        
        hf.create_dataset('FTCLE',      data=il)
        hf.create_dataset('FTLE',       data=FTLE)
        hf.create_dataset('LEs',        data=LEs)
        hf.create_dataset('state',      data=state)
        hf.create_dataset('Ntherm',     data=Ntherm)
        hf.create_dataset('norm_time',  data=norm_time)


    hf.close()